<a href="https://colab.research.google.com/github/Heba2424/Summery_Book_Generation/blob/main/Books_Scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import re
import time

In [ ]:
import requests
from bs4 import BeautifulSoup

# URL of the website with pagination
url = 'https://arab-books.com/books'

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'lxml')

# Find the pagination element (this might need to be adjusted based on the site's HTML structure)
pagination = soup.find('ul', class_='pagination')

# Find all the page number links within the pagination element
page_links = pagination.find_all('a')

# Extract the last page number (assuming it's the highest number)
last_page_number = max([int(link.text) for link in page_links if link.text.isdigit()])

print(f"Total number of pages: {last_page_number}")


Total number of pages: 719


In [ ]:
len(page_links)

4

## Extract Url's

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import time


# Possible class names to search for
book_div_classes = [
    "col-lg-3 col-md-4 col-6",
    "col-xxl-2 col-lg-3 col-md-4 col-6"
]



# List to store cleaned book titles
Books_urls = []
url_pattern = "(https://arab-books.com/books/([^\"]+)/)"
URl_RegEx = re.compile(url_pattern)

# Adjust the range if necessary after checking the website
for p in range(1, 201):
    url = f'https://arab-books.com/books/page/{p}/'
    print(f"Requesting URL: {url}")
    response = requests.get(url)

    if response.status_code == 200:
        print(f"Page {p} retrieved successfully")
        page = response.text
        Beautiful_page = BeautifulSoup(page, 'lxml')

        # Debug print to see if we are getting any content
        print(f"Parsing content on page {p}")

        book_divs = []
        for cls in book_div_classes:
          book_divs.extend(Beautiful_page.find_all('div', class_=cls))
        if not book_divs:
          print(f"No book divs found on page {p}. Continuing to next page.")
          continue




        for book_div in book_divs:
            myResult = URl_RegEx.search(str(book_div))

            if myResult:
                Books_urls.append(myResult.group())
            else:
                print(f"No url found in book div on page {p}")
    elif response.status_code == 404:
        print(f"Page {p} not found (status code 404). Stopping further requests.")
        break
    else:
        print(f"Failed to retrieve page {p}: status code {response.status_code}")

    # Sleep to avoid overwhelming the server
    time.sleep(1)

# Output all cleaned titles
for i, title in enumerate(Books_urls, start=1):
    print(f"{i}. {title}")


Streaming output truncated to the last 5000 lines.
Page 134 retrieved successfully
Parsing content on page 134
Requesting URL: https://arab-books.com/books/page/135/
Page 135 retrieved successfully
Parsing content on page 135
Requesting URL: https://arab-books.com/books/page/136/
Page 136 retrieved successfully
Parsing content on page 136
Requesting URL: https://arab-books.com/books/page/137/
Page 137 retrieved successfully
Parsing content on page 137
Requesting URL: https://arab-books.com/books/page/138/
Page 138 retrieved successfully
Parsing content on page 138
Requesting URL: https://arab-books.com/books/page/139/
Page 139 retrieved successfully
Parsing content on page 139
Requesting URL: https://arab-books.com/books/page/140/
Page 140 retrieved successfully
Parsing content on page 140
Requesting URL: https://arab-books.com/books/page/141/
Page 141 retrieved successfully
Parsing content on page 141
Requesting URL: https://arab-books.com/books/page/142/
Page 142 retrieved successful

In [ ]:
Books_urls[0]

'https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d8%a7%d9%84%d9%85%d8%a8%d8%a7%d8%af%d8%a6-%d8%a7%d9%84%d8%a3%d8%b3%d8%a7%d8%b3%d9%8a%d8%a9-%d9%84%d9%81%d9%87%d9%85-%d8%a7%d9%84%d9%82%d8%b1%d8%a2%d9%86-%d8%a7%d9%84%d9%83/'

In [ ]:
import pandas as pd



# Convert the list to a DataFrame
df_urls = pd.DataFrame(Books_urls, columns=["Books_Urls"])

# Save the DataFrame to a CSV file
df_urls.to_csv("urls_list.csv", index=False)

print("CSV file 'urls_list.csv' has been created.")


CSV file 'urls_list.csv' has been created.


## Extract Authors

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import time

# Function to extract author name from a book URL
def extract_author_name(book_url):
    try:
        response = requests.get(book_url)
        response.raise_for_status()  # Raise HTTPError for bad responses

        Beautiful_book = BeautifulSoup(response.text, 'lxml')
        details = Beautiful_book.find_all('li', class_="mb-2")

        # Extract the relevant 'li' element
        for detail in details:
            if 'المؤلف' in detail.text:
                # Convert the element to string for regex processing
                detail_str = str(detail)

                # Regular expression pattern to extract the author's name from an <a> tag
                pattern = r'<a[^>]*>([^<]+)</a>'
                match = re.search(pattern, detail_str)

                if match:
                    author_name = match.group(1).strip()
                    return author_name
                else:
                    return "not found"
        return "not found"
    except requests.RequestException as e:
        return f"Request failed: {e}"

# List to store the extracted author names
all_authors = []

# Process each URL sequentially
for i, book_url in enumerate(Books_urls):
    print(f"Processing URL {i + 1}/{len(Books_urls)}: {book_url}")
    author_name = extract_author_name(book_url)
    all_authors.append(author_name)
    print(f"Processed URL: {book_url} -> {author_name}")

    # Sleep to avoid overwhelming the server
    time.sleep(1)  # Adjust sleep time as needed

# Output all extracted author names
for i, author in enumerate(all_authors, start=1):
    print(f"{i}. {author}")


Streaming output truncated to the last 5000 lines.
Processing URL 4701/4800: https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d8%ac%d9%88%d9%87%d8%b1-%d8%a7%d9%84%d8%b9%d9%85%d9%84-pdf/
Processed URL: https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d8%ac%d9%88%d9%87%d8%b1-%d8%a7%d9%84%d8%b9%d9%85%d9%84-pdf/ -> ريوهو أوكاو
Processing URL 4702/4800: https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d8%a7%d9%84%d8%aa%d8%ba%d9%8a%d9%8a%d8%b1-%d8%a7%d9%84%d8%b0%d9%83%d9%8a-pdf-%d9%84%d9%84%d9%83%d8%a7%d8%aa%d8%a8-%d8%b9%d9%84%d9%8a-%d8%a7%d9%84%d8%ad%d9%85%d8%af%d9%8a/
Processed URL: https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d8%a7%d9%84%d8%aa%d8%ba%d9%8a%d9%8a%d8%b1-%d8%a7%d9%84%d8%b0%d9%83%d9%8a-pdf-%d9%84%d9%84%d9%83%d8%a7%d8%aa%d8%a8-%d8%b9%d9%84%d9%8a-%d8%a7%d9%84%d8%ad%d9%85%d8%af%d9%8a/ -> الكاتب علي الحمدي
Processing URL 4703/4800: https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d8%a7%d9%84%d8%aa%d8%a3%d9%85%d9%84-pdf/
Processed URL: https://arab

In [ ]:
import pandas as pd

# Convert the list to a DataFrame
df_authors = pd.DataFrame(all_authors, columns=["المؤلف"])

# Save the DataFrame to a CSV file with utf-8 encoding
df_authors.to_csv("authors_list.csv", index=False, encoding='utf-8-sig')

print("CSV file 'authors_list.csv' has been created.")


CSV file 'authors_list.csv' has been created.


## Extract Title Book

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import time

# Function to clean book titles by removing "PDF"
def clean_title(title):
    return re.sub(r'pdf', '', title, flags=re.IGNORECASE).strip()

# List to store cleaned book titles
all_books = []

# Define the range or use dynamic checking
start_page = 1
end_page = 201  # Start with a smaller range for testing

# Possible class names to search for
book_div_classes = [
    "col-lg-3 col-md-4 col-6",
    "col-xxl-2 col-lg-3 col-md-4 col-6"
]

for p in range(start_page, end_page):
    url = f'https://arab-books.com/books/page/{p}/'
    print(f"Requesting URL: {url}")
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError for bad responses

        if response.status_code == 200:
            print(f"Page {p} retrieved successfully")
            page = response.text
            Beautiful_page = BeautifulSoup(page, 'lxml')

            # Extract and print all div classes to debug
            all_divs = Beautiful_page.find_all('div')
            div_classes = [div.get('class') for div in all_divs]

            # Extract book divs using multiple class names
            book_divs = []
            for cls in book_div_classes:
                book_divs.extend(Beautiful_page.find_all('div', class_=cls))

            if not book_divs:
                print(f"No book divs found on page {p}. Continuing to next page.")
                continue

            # Extract titles from book divs
            for book_div in book_divs:
                title_tag = book_div.find('div', class_='book-cover-title fs-5 position-absolute top-50 mt-2 end-0 start-0 ms-3')
                if title_tag:
                    title = title_tag.text.strip()
                    cleaned_title = clean_title(title)
                    all_books.append(cleaned_title)
                else:
                    print(f"No title tag found in book div on page {p}")
        elif response.status_code == 404:
            print(f"Page {p} not found (status code 404). Stopping further requests.")
            break
        else:
            print(f"Failed to retrieve page {p}: status code {response.status_code}")
    except requests.RequestException as e:
        print(f"Request failed for page {p}: {e}")

    # Sleep to avoid overwhelming the server
    time.sleep(1)

# Output all cleaned titles
for i, title in enumerate(all_books, start=1):
    print(f"{i}. {title}")


Streaming output truncated to the last 5000 lines.
Requesting URL: https://arab-books.com/books/page/101/
Page 101 retrieved successfully
Requesting URL: https://arab-books.com/books/page/102/
Page 102 retrieved successfully
Requesting URL: https://arab-books.com/books/page/103/
Page 103 retrieved successfully
Requesting URL: https://arab-books.com/books/page/104/
Page 104 retrieved successfully
Requesting URL: https://arab-books.com/books/page/105/
Page 105 retrieved successfully
Requesting URL: https://arab-books.com/books/page/106/
Page 106 retrieved successfully
Requesting URL: https://arab-books.com/books/page/107/
Page 107 retrieved successfully
Requesting URL: https://arab-books.com/books/page/108/
Page 108 retrieved successfully
Requesting URL: https://arab-books.com/books/page/109/
Page 109 retrieved successfully
Requesting URL: https://arab-books.com/books/page/110/
Page 110 retrieved successfully
Requesting URL: https://arab-books.com/books/page/111/
Page 111 retrieved succe

In [ ]:
import pandas as pd
df_name = pd.DataFrame(all_books,columns= ['اسم الكتاب'])

# Save the DataFrame to a CSV file
df_name.to_csv("df_name.csv", index=False, encoding='utf-8-sig')

print("CSV file 'df_name.csv' has been created.")

CSV file 'df_name.csv' has been created.


In [ ]:
# Concatenate vertically
df = pd.concat([df_name, df_authors], axis=1)

df

,اسم الكتاب,المؤلف
0,كتاب المبادئ الأساسية لفهم القرآن الكريم,الكاتب أبو الأعلى المودودي
1,كتاب الحجاب,الكاتب أبو الأعلى المودودي
2,كتاب الخلافة والملك,الكاتب أبو الأعلى المودودي
3,كتاب مبادئ الإسلام,الكاتب أبو الأعلى المودودي
4,كتاب المصطلحات الأربعة في القرآن,الكاتب أبو الأعلى المودودي
...,...,...
4795,كتاب ترقية حياتك للكاتب بات ديفيلي,الكاتب بات ديفيلي
4796,كتاب أندورفين للكاتب سعيد النوبان,الكاتب سعيد النوبان
4797,كتاب إنسان جديد للكاتب مصطفى حسني,الداعية مصطفى حسني
4798,كتاب بإمكانك قراءة لغة الوجوه للكاتب ناعومي آ...,الكاتب ناعومي آر تيكل


In [ ]:
# Save the DataFrame to a CSV file
df.to_csv("df.csv", index=False, encoding='utf-8-sig')

print("CSV file 'df.csv' has been created.")

CSV file 'df.csv' has been created.


## Extract Pages

In [ ]:
response = requests.get('https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d9%84%d9%8a%d8%af%d8%a8%d8%b1%d9%88%d8%a7-%d8%a2%d9%8a%d8%a7%d8%aa%d9%87-pdf/')
response.raise_for_status()  # Raise HTTPError for bad responses

Beautiful_book = BeautifulSoup(response.text, 'lxml')
details = Beautiful_book.find_all('li', class_="mb-2")
details[3]

<li class="mb-2">
<span class="fw-bold">الحجم:</span>
10.18 </li>

In [ ]:
df1 = pd.read_csv('/content/urls_list.csv')
df1

,Books_Urls
0,https://arab-books.com/books/%d9%83%d8%aa%d8%a...
1,https://arab-books.com/books/%d9%83%d8%aa%d8%a...
2,https://arab-books.com/books/%d9%83%d8%aa%d8%a...
3,https://arab-books.com/books/%d9%83%d8%aa%d8%a...
4,https://arab-books.com/books/%d9%83%d8%aa%d8%a...
...,...
4795,https://arab-books.com/books/%d9%83%d8%aa%d8%a...
4796,https://arab-books.com/books/%d9%83%d8%aa%d8%a...
4797,https://arab-books.com/books/%d9%83%d8%aa%d8%a...
4798,https://arab-books.com/books/%d9%83%d8%aa%d8%a...


In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import time

# Function to extract the number of pages from a book URL
def extract_number_of_pages(book_url):
    try:
        response = requests.get(book_url)
        response.raise_for_status()  # Raise HTTPError for bad responses

        Beautiful_book = BeautifulSoup(response.text, 'lxml')
        details = Beautiful_book.find_all('li', class_="mb-2")

        number_of_pages = "not found"

        # Extract the number of pages from the relevant 'li' element
        for detail in details:
            if 'الصفحات' in detail.text:
                # Regular expression pattern to extract the number of pages
                pattern = r'الصفحات:\s*(\d+)'
                match = re.search(pattern, detail.text)

                if match:
                    number_of_pages = match.group(1).strip()
                break  # Stop after finding the number of pages

        return book_url, number_of_pages
    except requests.RequestException as e:
        return book_url, f"Request failed: {e}"

# List to store the details
all_pages = []

# Process each URL sequentially
for i, book_url in enumerate(df1['Books_Urls']):
    print(f"Processing URL {i + 1}/{len(df1['Books_Urls'])}: {book_url}")
    url, number_of_pages = extract_number_of_pages(book_url)
    all_pages.append( number_of_pages)
    print(f"Processed URL: {url} -> Pages: {number_of_pages}")

    # Sleep to avoid overwhelming the server
    time.sleep(1)  # Adjust sleep time as needed

# Output all extracted number of pages
for i, pages in enumerate(all_pages, start=1):
    print(f"{i}. URL: {url} -> Pages: {pages}")


Streaming output truncated to the last 5000 lines.
Processing URL 4701/4800: https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d8%ac%d9%88%d9%87%d8%b1-%d8%a7%d9%84%d8%b9%d9%85%d9%84-pdf/
Processed URL: https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d8%ac%d9%88%d9%87%d8%b1-%d8%a7%d9%84%d8%b9%d9%85%d9%84-pdf/ -> Pages: 163
Processing URL 4702/4800: https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d8%a7%d9%84%d8%aa%d8%ba%d9%8a%d9%8a%d8%b1-%d8%a7%d9%84%d8%b0%d9%83%d9%8a-pdf-%d9%84%d9%84%d9%83%d8%a7%d8%aa%d8%a8-%d8%b9%d9%84%d9%8a-%d8%a7%d9%84%d8%ad%d9%85%d8%af%d9%8a/
Processed URL: https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d8%a7%d9%84%d8%aa%d8%ba%d9%8a%d9%8a%d8%b1-%d8%a7%d9%84%d8%b0%d9%83%d9%8a-pdf-%d9%84%d9%84%d9%83%d8%a7%d8%aa%d8%a8-%d8%b9%d9%84%d9%8a-%d8%a7%d9%84%d8%ad%d9%85%d8%af%d9%8a/ -> Pages: 122
Processing URL 4703/4800: https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d8%a7%d9%84%d8%aa%d8%a3%d9%85%d9%84-pdf/
Processed URL: https://arab-books.c

In [ ]:
import pandas as pd
df_pages = pd.DataFrame(all_pages,columns=['عدد الصفحات'])
df_pages

# Save the DataFrame to a CSV file
df_pages.to_csv("df_pages.csv", index=False, encoding='utf-8-sig')

print("CSV file 'df_pages.csv' has been created.")

CSV file 'df_pages.csv' has been created.


In [ ]:
df = pd.read_csv('/content/df.csv')
df.head()

,اسم الكتاب,المؤلف
0,كتاب المبادئ الأساسية لفهم القرآن الكريم,الكاتب أبو الأعلى المودودي
1,كتاب الحجاب,الكاتب أبو الأعلى المودودي
2,كتاب الخلافة والملك,الكاتب أبو الأعلى المودودي
3,كتاب مبادئ الإسلام,الكاتب أبو الأعلى المودودي
4,كتاب المصطلحات الأربعة في القرآن,الكاتب أبو الأعلى المودودي


In [ ]:
# Concatenate vertically
df = pd.concat([df, df_pages], axis=1)

df

,اسم الكتاب,المؤلف,عدد الصفحات
0,كتاب المبادئ الأساسية لفهم القرآن الكريم,الكاتب أبو الأعلى المودودي,38
1,كتاب الحجاب,الكاتب أبو الأعلى المودودي,344
2,كتاب الخلافة والملك,الكاتب أبو الأعلى المودودي,246
3,كتاب مبادئ الإسلام,الكاتب أبو الأعلى المودودي,79
4,كتاب المصطلحات الأربعة في القرآن,الكاتب أبو الأعلى المودودي,139
...,...,...,...
4795,كتاب ترقية حياتك للكاتب بات ديفيلي,الكاتب بات ديفيلي,152
4796,كتاب أندورفين للكاتب سعيد النوبان,الكاتب سعيد النوبان,122
4797,كتاب إنسان جديد للكاتب مصطفى حسني,الداعية مصطفى حسني,334
4798,كتاب بإمكانك قراءة لغة الوجوه للكاتب ناعومي آ...,الكاتب ناعومي آر تيكل,267


In [ ]:
# Save the DataFrame to a CSV file
df.to_csv("datast.csv", index=False, encoding='utf-8-sig')

print("CSV file 'datast.csv' has been created.")

CSV file 'datast.csv' has been created.


## Extract Size

In [ ]:
df1 = pd.read_csv('/content/urls_list (1).csv')
df1.head()

,Books_Urls
0,https://arab-books.com/books/%d9%83%d8%aa%d8%a...
1,https://arab-books.com/books/%d9%83%d8%aa%d8%a...
2,https://arab-books.com/books/%d9%83%d8%aa%d8%a...
3,https://arab-books.com/books/%d9%83%d8%aa%d8%a...
4,https://arab-books.com/books/%d9%83%d8%aa%d8%a...


In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import time

# Function to extract the size from a book URL
def extract_size(book_url):
    try:
        response = requests.get(book_url)
        response.raise_for_status()  # Raise HTTPError for bad responses

        Beautiful_book = BeautifulSoup(response.text, 'lxml')
        details = Beautiful_book.find_all('li', class_="mb-2")

        size = "not found"

        # Extract the size from the relevant 'li' element
        for detail in details:
            if 'الحجم' in detail.text:
                # Regular expression pattern to extract the size
                pattern = r'الحجم:\s*([\d.]+)'
                match = re.search(pattern, detail.text)

                if match:
                    size = match.group(1).strip()
                break  # Stop after finding the size

        return size
    except requests.RequestException as e:
        return f"Request failed: {e}"

# List to store the details
all_sizes = []

# Process each URL sequentially
for i, book_url in enumerate(df1['Books_Urls']):
    print(f"Processing URL {i + 1}/{len(df1['Books_Urls'])}: {book_url}")
    size = extract_size(book_url)
    all_sizes.append(size)
    print(f"Processed URL -> Size: {size}")

    # Sleep to avoid overwhelming the server
    time.sleep(1)  # Adjust sleep time as needed

# Output all extracted sizes
for i, size in enumerate(all_sizes, start=1):
    print(f"{i}. Size: {size}")


Streaming output truncated to the last 5000 lines.
Processing URL 4701/4800: https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d8%ac%d9%88%d9%87%d8%b1-%d8%a7%d9%84%d8%b9%d9%85%d9%84-pdf/
Processed URL -> Size: 1
Processing URL 4702/4800: https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d8%a7%d9%84%d8%aa%d8%ba%d9%8a%d9%8a%d8%b1-%d8%a7%d9%84%d8%b0%d9%83%d9%8a-pdf-%d9%84%d9%84%d9%83%d8%a7%d8%aa%d8%a8-%d8%b9%d9%84%d9%8a-%d8%a7%d9%84%d8%ad%d9%85%d8%af%d9%8a/
Processed URL -> Size: 8
Processing URL 4703/4800: https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d8%a7%d9%84%d8%aa%d8%a3%d9%85%d9%84-pdf/
Processed URL -> Size: 5.4
Processing URL 4704/4800: https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d8%a7%d9%84%d8%aa%d9%81%d9%83%d9%8a%d8%b1-%d8%ae%d8%a7%d8%b1%d8%ac-%d8%a7%d9%84%d8%b5%d9%86%d8%af%d9%88%d9%82-pdf/
Processed URL -> Size: 4.7
Processing URL 4705/4800: https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d8%a7%d9%84%d8%aa%d9%86%d8%a8%d9%8a%d9%87-pdf/
Proces

In [ ]:
import pandas as pd
df_sizes = pd.DataFrame(all_sizes,columns=['الحجم'])
df_sizes

# Save the DataFrame to a CSV file
df_sizes.to_csv("df_sizes.csv", index=False, encoding='utf-8-sig')

print("CSV file 'df_sizes.csv' has been created.")

CSV file 'df_sizes.csv' has been created.


In [ ]:
df = pd.read_csv('/content/datast.csv')
df.head()

In [ ]:
# Concatenate vertically
df = pd.concat([df, df_sizes], axis=1)

df

In [ ]:
# Save the DataFrame to a CSV file
df.to_csv("datast.csv", index=False, encoding='utf-8-sig')

print("CSV file 'datast.csv' has been created.")

## Extract Language

In [ ]:
response = requests.get('https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d9%84%d9%8a%d8%af%d8%a8%d8%b1%d9%88%d8%a7-%d8%a2%d9%8a%d8%a7%d8%aa%d9%87-pdf/')
response.raise_for_status()  # Raise HTTPError for bad responses

Beautiful_book = BeautifulSoup(response.text, 'lxml')
details = Beautiful_book.find_all('li', class_="mb-2")
details[4]

<li class="mb-2">
<span class="fw-bold">اللغة:</span>
العربية </li>

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

# Function to extract the language from a book URL
def extract_language(book_url):
    try:
        response = requests.get(book_url)
        response.raise_for_status()  # Raise HTTPError for bad responses

        Beautiful_book = BeautifulSoup(response.text, 'lxml')
        details = Beautiful_book.find_all('li', class_="mb-2")

        language = "not found"

        # Extract the language from the relevant 'li' element
        for detail in details:
            if 'اللغة' in detail.text:
                # Regular expression pattern to extract the language
                pattern = r'اللغة:\s*(\S+)'
                match = re.search(pattern, detail.text)

                if match:
                    language = match.group(1).strip()
                break  # Stop after finding the language

        return language
    except requests.RequestException as e:
        return f"Request failed: {e}"

# Initialize an empty DataFrame
df_languages = pd.DataFrame(columns=['اللغة'])

# Process each URL sequentially
for i, book_url in enumerate(df1['Books_Urls']):
    print(f"Processing URL {i + 1}/{len(df1['Books_Urls'])}: {book_url}")
    language = extract_language(book_url)
    df_languages = pd.concat([df_languages, pd.DataFrame({'اللغة': [language]})], ignore_index=True)
    print(f"Processed URL -> Language: {language}")

    # Sleep to avoid overwhelming the server
    time.sleep(1)  # Adjust sleep time as needed

# Output the DataFrame
print(df_languages)


In [ ]:
df_languages

In [ ]:
# Save the DataFrame to a CSV file
df_languages.to_csv("df_languages.csv", index=False, encoding='utf-8-sig')

print("CSV file 'df_languages.csv' has been created.")

In [ ]:
df = pd.read_csv('/content/datast.csv')
df

In [ ]:

# Concatenate vertically
df = pd.concat([df, df_languages], axis=1)

df

In [ ]:
# Save the DataFrame to a CSV file
df.to_csv("datast.csv", index=False, encoding='utf-8-sig')

print("CSV file 'datast.csv' has been created.")

## Extract Type

In [ ]:
response = requests.get('https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d9%84%d9%8a%d8%af%d8%a8%d8%b1%d9%88%d8%a7-%d8%a2%d9%8a%d8%a7%d8%aa%d9%87-pdf/')
response.raise_for_status()  # Raise HTTPError for bad responses

Beautiful_book = BeautifulSoup(response.text, 'lxml')
details = Beautiful_book.find_all('li', class_="mb-2")
details[5]

<li class="mb-2">
<span class="fw-bold">النوع:</span>
pdf </li>

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

# Function to extract the type from a book URL
def extract_type(book_url):
    try:
        response = requests.get(book_url)
        response.raise_for_status()  # Raise HTTPError for bad responses

        Beautiful_book = BeautifulSoup(response.text, 'lxml')
        details = Beautiful_book.find_all('li', class_="mb-2")

        book_type = "not found"

        # Extract the type from the relevant 'li' element
        for detail in details:
            if 'النوع' in detail.text:
                # Regular expression pattern to extract the type
                pattern = r'النوع:\s*(\S+)'
                match = re.search(pattern, detail.text)

                if match:
                    book_type = match.group(1).strip()
                break  # Stop after finding the type

        return book_type
    except requests.RequestException as e:
        return f"Request failed: {e}"

# Initialize an empty DataFrame
df_types = pd.DataFrame(columns=['النوع'])

# Process each URL sequentially
for i, book_url in enumerate(df1['Books_Urls']):
    print(f"Processing URL {i + 1}/{len(df1['Books_Urls'])}: {book_url}")
    book_type = extract_type(book_url)
    df_types = pd.concat([df_types, pd.DataFrame({'النوع': [book_type]})], ignore_index=True)
    print(f"Processed URL -> Type: {book_type}")

    # Sleep to avoid overwhelming the server
    time.sleep(1)  # Adjust sleep time as needed

# Output the DataFrame
df_types


In [ ]:
df_types

In [ ]:
# Save the DataFrame to a CSV file
df_types.to_csv("df_types.csv", index=False, encoding='utf-8-sig')

print("CSV file 'df_types.csv' has been created.")

In [ ]:
df = pd.read_csv('/content/datast.csv')
df.head()

In [ ]:
# Concatenate vertically
df = pd.concat([df, df_types], axis=1)

df

In [ ]:
# Save the DataFrame to a CSV file
df.to_csv("datast.csv", index=False, encoding='utf-8-sig')

print("CSV file 'datast.csv' has been created.")

## Extract Section

In [ ]:
df1 = pd.read_csv('/content/urls_list (1).csv')
df1.head()

,Books_Urls
0,https://arab-books.com/books/%d9%83%d8%aa%d8%a...
1,https://arab-books.com/books/%d9%83%d8%aa%d8%a...
2,https://arab-books.com/books/%d9%83%d8%aa%d8%a...
3,https://arab-books.com/books/%d9%83%d8%aa%d8%a...
4,https://arab-books.com/books/%d9%83%d8%aa%d8%a...


In [ ]:
response = requests.get('https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d9%84%d9%8a%d8%af%d8%a8%d8%b1%d9%88%d8%a7-%d8%a2%d9%8a%d8%a7%d8%aa%d9%87-pdf/')
response.raise_for_status()  # Raise HTTPError for bad responses

Beautiful_book = BeautifulSoup(response.text, 'lxml')
details = Beautiful_book.find_all('li', class_="mb-2")
details[6]

<li class="mb-2">
<span class="fw-bold">الاقسام:</span>
<a class="text-muted" href="https://arab-books.com/section/%d8%aa%d8%ad%d9%85%d9%8a%d9%84-%d9%83%d8%aa%d8%a8-%d8%a7%d8%b3%d9%84%d8%a7%d9%85%d9%8a%d8%a9-pdf/">كتب اسلامية</a><span class="last-child">,</span> </li>

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

# Function to extract the section from a book URL
def extract_section(book_url):
    try:
        response = requests.get(book_url)
        response.raise_for_status()  # Raise HTTPError for bad responses

        Beautiful_book = BeautifulSoup(response.text, 'lxml')
        details = Beautiful_book.find_all('li', class_="mb-2")

        section = "not found"

        # Extract the section from the relevant 'li' element
        for detail in details:
            if 'الاقسام' in detail.text:
                # Regular expression pattern to extract the section
                pattern = r'الاقسام:\s*(.*)'
                match = re.search(pattern, detail.text)

                if match:
                    section = match.group(1).strip()
                    # Remove HTML tags from the section string
                    section = BeautifulSoup(section, "lxml").text.strip()
                break  # Stop after finding the section

        return section
    except requests.RequestException as e:
        return f"Request failed: {e}"

# Initialize an empty DataFrame
df_sections = pd.DataFrame(columns=['الاقسام'])

# Process each URL sequentially
for i, book_url in enumerate(df1['Books_Urls']):
    print(f"Processing URL {i + 1}/{len(df1['Books_Urls'])}: {book_url}")
    section = extract_section(book_url)
    df_sections = pd.concat([df_sections, pd.DataFrame({'الاقسام': [section]})], ignore_index=True)
    print(f"Processed URL -> Section: {section}")

    # Sleep to avoid overwhelming the server
    time.sleep(1)  # Adjust sleep time as needed

# Output the DataFrame
print(df_sections)


Streaming output truncated to the last 5000 lines.
Processing URL 2308/4800: https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d9%86%d8%b8%d9%85-%d8%a7%d9%84%d8%ac%d9%88%d9%87%d8%b1%d8%a9-%d8%a7%d9%84%d9%81%d8%b1%d9%8a%d8%af%d8%a9-%d9%81%d9%89-%d8%aa%d8%ad%d9%82%d9%8a%d9%82-%d8%a7%d9%84%d8%b9%d9%82/
Processed URL -> Section: كتب العقيدة,
Processing URL 2309/4800: https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d8%af%d9%84%d9%8a%d9%84-%d8%a3%d8%b1%d8%a8%d8%a7%d8%a8-%d8%a7%d9%84%d9%81%d9%84%d8%a7%d8%ad-pdf/
Processed URL -> Section: كتب الحديث الشريف,
Processing URL 2310/4800: https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d9%85%d9%81%d8%aa%d8%a7%d8%ad-%d8%af%d8%a7%d8%b1-%d8%a7%d9%84%d8%b3%d9%84%d8%a7%d9%85-pdf/
Processed URL -> Section: كتب اسلامية,
Processing URL 2311/4800: https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d9%85%d8%ac%d9%85%d9%84-%d8%aa%d8%a7%d8%b1%d9%8a%d8%ae-%d8%a7%d9%84%d8%a3%d9%86%d8%af%d9%84%d8%b3-pdf-2/
Processed URL -> Section: كتب التار

In [ ]:
df_sections

,الاقسام
0,"كتب علوم القرآن,"
1,"قضايا,"
2,"كتب متنوعة,"
3,"كتب اسلامية,"
4,"كتب تفسير القرآن الكريم,"
...,...
4795,"كتب التنمية البشرية,"
4796,"كتب التنمية البشرية,"
4797,"كتب التنمية البشرية,"
4798,"كتب التنمية البشرية,"


In [ ]:
# Save the DataFrame to a CSV file
df_sections.to_csv("df_sections.csv", index=False, encoding='utf-8-sig')

print("CSV file 'df_sections.csv' has been created.")

CSV file 'df_sections.csv' has been created.


In [ ]:
# Concatenate vertically
df = pd.concat([df, df_sections], axis=1)

df

,اسم الكتاب,المؤلف,عدد الصفحات,الحجم,اللغة,الاقسام
0,كتاب المبادئ الأساسية لفهم القرآن الكريم,الكاتب أبو الأعلى المودودي,38,1.18,العربية,"كتب علوم القرآن,"
1,كتاب الحجاب,الكاتب أبو الأعلى المودودي,344,4.87,العربية,"قضايا,"
2,كتاب الخلافة والملك,الكاتب أبو الأعلى المودودي,246,7.94,العربية,"كتب متنوعة,"
3,كتاب مبادئ الإسلام,الكاتب أبو الأعلى المودودي,79,524.45,العربية,"كتب اسلامية,"
4,كتاب المصطلحات الأربعة في القرآن,الكاتب أبو الأعلى المودودي,139,2.71,العربية,"كتب تفسير القرآن الكريم,"
...,...,...,...,...,...,...
4795,كتاب ترقية حياتك للكاتب بات ديفيلي,الكاتب بات ديفيلي,152,2.2,عربي,"كتب التنمية البشرية,"
4796,كتاب أندورفين للكاتب سعيد النوبان,الكاتب سعيد النوبان,122,6,عربي,"كتب التنمية البشرية,"
4797,كتاب إنسان جديد للكاتب مصطفى حسني,الداعية مصطفى حسني,334,5.4,عربي,"كتب التنمية البشرية,"
4798,كتاب بإمكانك قراءة لغة الوجوه للكاتب ناعومي آ...,الكاتب ناعومي آر تيكل,267,5,عربي,"كتب التنمية البشرية,"


In [ ]:
# Save the DataFrame to a CSV file
df.to_csv("datast.csv", index=False, encoding='utf-8-sig')

print("CSV file 'datast.csv' has been created.")

CSV file 'datast.csv' has been created.


## Extract Summary

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Assuming df1 is your DataFrame and 'Books_Urls' is the column with the URLs

# Function to extract the summaries from a given URL
def extract_summaries(book_url):
    try:
        # Send the HTTP request and parse the content
        result = requests.get(book_url)
        result.raise_for_status()  # Raise an error for bad responses
        soup = BeautifulSoup(result.content, 'lxml')

        # Find all div elements with the specified class
        div_elements = soup.find_all('div', class_='mb-3 border rounded p-md-3 p-2')

        # Check if the specified div exists (assuming the third div is intended)
        if len(div_elements) > 2:
            # Extract and concatenate the text from the third div element
            summary_text = div_elements[2].text.strip()
            # Replace multiple spaces with a single space
            summary_text = re.sub(r'\s+', ' ', summary_text)
            return summary_text
        else:
            return "not found"
    except requests.RequestException as e:
        return f"Request failed: {e}"

# Apply the extraction function to all URLs and store the results in a new column
summaries = []
for i, url in enumerate(df1['Books_Urls'], start=1):
    print(f"Processing URL {i}/{len(df1['Books_Urls'])}: {url}")
    summary = extract_summaries(url)
    summaries.append(summary)
    print(f"Summary for URL {i}: {summary}\n")

df1['Summaries'] = summaries

# Output the DataFrame with the new column
print(df1[['Books_Urls', 'Summaries']])


In [ ]:
df1.drop('Books_Urls', axis=1, inplace=True)
df1

In [ ]:
# Save the DataFrame to a CSV file
df1.to_csv("summery.csv", index=False, encoding='utf-8-sig')

print("CSV file 'summery.csv' has been created.")

In [ ]:
df = pd.read_csv('/content/datast.csv')
df.head()

,اسم الكتاب,المؤلف,عدد الصفحات,الحجم,اللغة,الاقسام
0,كتاب المبادئ الأساسية لفهم القرآن الكريم,الكاتب أبو الأعلى المودودي,38,1.18,العربية,"كتب علوم القرآن,"
1,كتاب الحجاب,الكاتب أبو الأعلى المودودي,344,4.87,العربية,"قضايا,"
2,كتاب الخلافة والملك,الكاتب أبو الأعلى المودودي,246,7.94,العربية,"كتب متنوعة,"
3,كتاب مبادئ الإسلام,الكاتب أبو الأعلى المودودي,79,524.45,العربية,"كتب اسلامية,"
4,كتاب المصطلحات الأربعة في القرآن,الكاتب أبو الأعلى المودودي,139,2.71,العربية,"كتب تفسير القرآن الكريم,"


In [ ]:
sry = pd.read_csv('/content/summery (1).csv')
sry.head()

,Summaries
0,كتاب المبادئ الأساسية لفهم القرآن الكريم PDF ت...
1,كتاب الحجاب PDF تلخيص كتاب الحجاب PDF للكاتب أ...
2,كتاب الخلافة والملك PDF تلخيص كتاب الخلافة وال...
3,كتاب مبادئ الإسلام PDF تلخيص كتاب مبادئ الإسلا...
4,كتاب المصطلحات الأربعة في القرآن PDF تلخيص كتا...


In [ ]:
# Concatenate vertically
df = pd.concat([df, sry], axis=1)

df

,اسم الكتاب,المؤلف,عدد الصفحات,الحجم,اللغة,الاقسام,النوع,Summaries
0,كتاب المبادئ الأساسية لفهم القرآن الكريم,الكاتب أبو الأعلى المودودي,38,1.18,العربية,"كتب علوم القرآن,",pdf,كتاب المبادئ الأساسية لفهم القرآن الكريم PDF ت...
1,كتاب الحجاب,الكاتب أبو الأعلى المودودي,344,4.87,العربية,"قضايا,",pdf,كتاب الحجاب PDF تلخيص كتاب الحجاب PDF للكاتب أ...
2,كتاب الخلافة والملك,الكاتب أبو الأعلى المودودي,246,7.94,العربية,"كتب متنوعة,",pdf,كتاب الخلافة والملك PDF تلخيص كتاب الخلافة وال...
3,كتاب مبادئ الإسلام,الكاتب أبو الأعلى المودودي,79,524.45,العربية,"كتب اسلامية,",pdf,كتاب مبادئ الإسلام PDF تلخيص كتاب مبادئ الإسلا...
4,كتاب المصطلحات الأربعة في القرآن,الكاتب أبو الأعلى المودودي,139,2.71,العربية,"كتب تفسير القرآن الكريم,",not found,كتاب المصطلحات الأربعة في القرآن PDF تلخيص كتا...
...,...,...,...,...,...,...,...,...
4795,كتاب ترقية حياتك للكاتب بات ديفيلي,الكاتب بات ديفيلي,152,2.2,عربي,"كتب التنمية البشرية,",pdf,كتاب ترقية حياتك PDF للكاتب بات ديفيلي تلخيص ك...
4796,كتاب أندورفين للكاتب سعيد النوبان,الكاتب سعيد النوبان,122,6,عربي,"كتب التنمية البشرية,",pdf,كتاب أندورفين PDF للكاتب سعيد النوبان تلخيص كت...
4797,كتاب إنسان جديد للكاتب مصطفى حسني,الداعية مصطفى حسني,334,5.4,عربي,"كتب التنمية البشرية,",pdf,كتاب إنسان جديد PDF للكاتب مصطفى حسني تلخيص كت...
4798,كتاب بإمكانك قراءة لغة الوجوه للكاتب ناعومي آ...,الكاتب ناعومي آر تيكل,267,5,عربي,"كتب التنمية البشرية,",pdf,كتاب بإمكانك قراءة لغة الوجوه PDF للكاتب ناعوم...


In [ ]:
# Save the DataFrame to a CSV file
df.to_csv("datast.csv", index=False, encoding='utf-8-sig')

print("CSV file 'summery.csv' has been created.")

CSV file 'summery.csv' has been created.


In [ ]:
# Count occurrences of "not found" in the entire DataFrame
not_found_count = (df == "not found").sum().sum()

print(f"Number of 'not found' entries in the DataFrame: {not_found_count}")


Number of 'not found' entries in the DataFrame: 1752


In [ ]:
# Step 1: Create a boolean mask where any column contains "not found"
mask = (df == "not found").any(axis=1)

# Step 2: Use this mask to filter the DataFrame and drop the rows
df_filtered = df[~mask]

# Step 3: Reset the index of the resulting DataFrame
df_filtered.reset_index(drop=True, inplace=True)

# Print the filtered DataFrame
print("\nFiltered DataFrame (rows with 'not found' dropped):")
df_filtered


Original DataFrame:

Filtered DataFrame (rows with 'not found' dropped):


,اسم الكتاب,المؤلف,عدد الصفحات,الحجم,اللغة,الاقسام,النوع,Summaries
0,كتاب المبادئ الأساسية لفهم القرآن الكريم,الكاتب أبو الأعلى المودودي,38,1.18,العربية,"كتب علوم القرآن,",pdf,كتاب المبادئ الأساسية لفهم القرآن الكريم PDF ت...
1,كتاب الحجاب,الكاتب أبو الأعلى المودودي,344,4.87,العربية,"قضايا,",pdf,كتاب الحجاب PDF تلخيص كتاب الحجاب PDF للكاتب أ...
2,كتاب الخلافة والملك,الكاتب أبو الأعلى المودودي,246,7.94,العربية,"كتب متنوعة,",pdf,كتاب الخلافة والملك PDF تلخيص كتاب الخلافة وال...
3,كتاب مبادئ الإسلام,الكاتب أبو الأعلى المودودي,79,524.45,العربية,"كتب اسلامية,",pdf,كتاب مبادئ الإسلام PDF تلخيص كتاب مبادئ الإسلا...
4,كتاب الفلسفة الرحمانية عند زكي الأرسوزي,دكتور محمود محمد علي,29,1,العربية,"كتب الفلسفة المعاصرة,",pdf,كتاب الفلسفة الرحمانية عند زكي الأرسوزي PDF تل...
...,...,...,...,...,...,...,...,...
3807,كتاب ترقية حياتك للكاتب بات ديفيلي,الكاتب بات ديفيلي,152,2.2,عربي,"كتب التنمية البشرية,",pdf,كتاب ترقية حياتك PDF للكاتب بات ديفيلي تلخيص ك...
3808,كتاب أندورفين للكاتب سعيد النوبان,الكاتب سعيد النوبان,122,6,عربي,"كتب التنمية البشرية,",pdf,كتاب أندورفين PDF للكاتب سعيد النوبان تلخيص كت...
3809,كتاب إنسان جديد للكاتب مصطفى حسني,الداعية مصطفى حسني,334,5.4,عربي,"كتب التنمية البشرية,",pdf,كتاب إنسان جديد PDF للكاتب مصطفى حسني تلخيص كت...
3810,كتاب بإمكانك قراءة لغة الوجوه للكاتب ناعومي آ...,الكاتب ناعومي آر تيكل,267,5,عربي,"كتب التنمية البشرية,",pdf,كتاب بإمكانك قراءة لغة الوجوه PDF للكاتب ناعوم...


In [ ]:
# Save the DataFrame to a CSV file
df_filtered.to_csv("dataset.csv", index=False, encoding='utf-8-sig')

print("CSV file 'dataset.csv' has been created.")

CSV file 'dataset.csv' has been created.
